<a href="https://colab.research.google.com/github/kuberiitb/retail_price_ai_agent/blob/main/notebooks/RetailPriceAgent_Text2Sql_V3withSupervisor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Agent created using code from [LangGraph Tutorial](https://python.langchain.com/docs/tutorials/sql_qa/)

In [2]:
!pip install langchain langchain_community langchain-openai langgraph ipython --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 16.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


# Setting up SQL data

In [3]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///retail_price_agent.db")
print(db.get_usable_table_names())

['competitior_information', 'current_product_information', 'forecast_data', 'historical_data', 'inventory_data']


In [4]:
for table_name in db.get_usable_table_names():
  print(table_name)
  print(db.get_table_info([table_name]))
  print("="*80)

competitior_information

CREATE TABLE competitior_information (
	sku_id INTEGER, 
	product_name TEXT, 
	category TEXT, 
	unit_price REAL, 
	promotion TEXT, 
	discount_pct REAL
)

/*
3 rows from competitior_information table:
sku_id	product_name	category	unit_price	promotion	discount_pct
1	t-shirt	men	864.5	BTGO	0.3
8	t-shirt	women	946.4	BTGO	0.3
*/
current_product_information

CREATE TABLE current_product_information (
	sku_id INTEGER, 
	base_price REAL, 
	base_demand REAL, 
	elasticity REAL, 
	margin REAL
)

/*
3 rows from current_product_information table:
sku_id	base_price	base_demand	elasticity	margin
1	1460.0	500.0	-1.06	0.94
8	1820.0	800.0	-0.93	1.08
*/
forecast_data

CREATE TABLE forecast_data (
	sku_id INTEGER, 
	product_name TEXT, 
	category TEXT, 
	date TIMESTAMP, 
	units_sale REAL
)

/*
3 rows from forecast_data table:
sku_id	product_name	category	date	units_sale
1	t-shirt	men	2025-09-01 00:00:00	41.0
1	t-shirt	men	2025-10-01 00:00:00	41.0
1	t-shirt	men	2025-11-01 00:00:00	4

In [5]:
# Langgraph State
from typing_extensions import TypedDict

class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str

# Setup LLM

In [6]:
import os
from dotenv import load_dotenv
load_dotenv(".env")

True

In [7]:
# from langchain.chat_models import ChatOpenAI

if not os.environ["OPENAI_API_KEY"]:
    os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [8]:
llm.invoke("Hi").content

'Hello! How can I assist you today?'

# Using inbuilt Langgraph ReAct agent

In [9]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

for t in tools:
  print(t.name, ":", t.description)

sql_db_query : Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.
sql_db_schema : Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3
sql_db_list_tables : Input is an empty string, output is a comma-separated list of tables in the database.
sql_db_query_checker : Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!


In [10]:
database_information = """"
TABLE: historical_data - Contains historical monthly-level sales, pricing, and profit data for each SKU.

COLUMN: sku_id - Unique identifier for each product-category combination.

COLUMN: product_name - Name of the specific product.

COLUMN: category - Category or department the product belongs to.

COLUMN: date - Month (or month-start date) representing the sales period.

COLUMN: unit_price - Actual selling price per unit after applying discounts.

COLUMN: unit_cost - Cost to produce or acquire one unit of the product.

COLUMN: discount_pct - Percentage discount applied to the base price.

COLUMN: seasonality_factor - Seasonal adjustment factor reflecting demand fluctuations.

COLUMN: units_sold - Total quantity sold for the SKU during the given period.

COLUMN: revenue - Total revenue generated = unit_price * units_sold.

COLUMN: profit - Total profit = (unit_price - unit_cost) * units_sold.

#########################################################################################

TABLE: current_product_information - Stores reference or metadata information about each of our SKU used for pricing and forecasting.

COLUMN: sku_id - Unique identifier for each product-category combination.

COLUMN: base_price - Reference or standard list price of the product.

COLUMN: base_demand - Baseline expected demand level for the product.

COLUMN: elasticity - Price elasticity coefficient indicating sensitivity of demand to price changes.

COLUMN: margin - Target profit margin ratio derived from elasticity or business rules.

#########################################################################################
TABLE: forecast_data - Contains forecasted monthly unit sales for each SKU based on predictive modeling.

COLUMN: sku_id - Unique identifier for each product-category combination.

COLUMN: product_name - Name of the specific product.

COLUMN: category - Category or department the product belongs to.

COLUMN: date - Forecast month or future period.

COLUMN: units_sale - Forecasted number of units expected to be sold.

#########################################################################################

TABLE: inventory_data - Tracks current stock levels for each SKU in the inventory.

COLUMN: sku_id - Unique identifier for each product-category combination.

COLUMN: product_name - Name of the specific product.

COLUMN: category - Category or department the product belongs to.

COLUMN: stock - Current quantity of the SKU available in inventory.

#########################################################################################

TABLE: competitior_information - Captures competitor pricing and promotion details for comparative analysis.

COLUMN: sku_id - Unique identifier representing the same or equivalent SKU.

COLUMN: product_name - Name of the product for cross-reference with competitors.

COLUMN: category - Product category for comparison.

COLUMN: unit_price - Competitor’s selling price for the product.

COLUMN: promotion - Competitor’s promotion or offer label (e.g., "BOGO","NONE" or discount value).

COLUMN: discount_pct - Discount percentage applied by the competitor, if available.
"""

In [11]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from langchain.memory import ConversationBufferMemory

extra_details_about_data = """
1. SKU means product_name+category combination, can be represented by sku_id
2. product means product_name unless specified otherwise.
3. While returning SKU information, mention its details like product_name and category.
4. All the price and revenue information is in INR.
"""

system_message = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. Unless the user
specifies a specific number of examples they wish to obtain, always limit your
query to at most {top_k} results.

{extra_details_about_data}

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

To start you should ALWAYS look at the tables in the database to see what you
can query. Do NOT skip this step.

Use this description of tables and columns for reference.
{database_information}

Then you should query the schema of the most relevant tables.

Business context:
Product usually means product-category combination.
""".format(
    dialect="SQLite",
    top_k=5,
    database_information=database_information,
    extra_details_about_data=extra_details_about_data
)

data_agent = create_react_agent(
    name="Retail_Data_Agent",
    model=llm,
    tools=tools,
    prompt=system_message,
)

In [12]:
question = "Which is highest selling SKU?"
data_agent.invoke({"messages": [{"role": "user", "content": question}]})

{'messages': [HumanMessage(content='Which is highest selling SKU?', additional_kwargs={}, response_metadata={}, id='85a80b20-0c37-43d7-a487-3e78f06a85d2'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8SWDhudYWzE0xbecxKzRPHdA', 'function': {'arguments': '{}', 'name': 'sql_db_list_tables'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 1139, 'total_tokens': 1151, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CQntgK0o6YTt7BIAwmLSRIqzq7bOG', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='Retail_Data_Agent', id='run--897f19e4-eb84-40eb-97de-45ccc02b209d-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {

## Example with pretty printing

In [ ]:
question = "Which is highest selling SKU_ID?"

for step in data_agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Which is highest selling SKU_ID?
================================== Ai Message ==================================
Name: Retail_Data_Agent
Tool Calls:
  sql_db_list_tables (call_dnY3Mjv4NHrWlmQBSYCFh1ex)
 Call ID: call_dnY3Mjv4NHrWlmQBSYCFh1ex
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

competitior_information, current_product_information, forecast_data, historical_data, inventory_data
================================== Ai Message ==================================
Name: Retail_Data_Agent
Tool Calls:
  sql_db_schema (call_XqAkkawm2pft9vGKrW7EiU9m)
 Call ID: call_XqAkkawm2pft9vGKrW7EiU9m
  Args:
    table_names: historical_data
  sql_db_schema (call_NVbwMqXBP2uH5DXTeXrs5Lnq)
 Call ID: call_NVbwMqXBP2uH5DXTeXrs5Lnq
  Args:
    table_names: current_product_information
================================= Tool Message =========================

## Returning final output only

## skipping all the intermediate steps

In [13]:
from IPython.display import display, Markdown

In [19]:
def get_sql_reponse(question, verbose=False):
  for step in data_agent.stream(
      {"messages": [{"role": "user", "content": question}]},
      stream_mode="values",
  ):
      if verbose:
        step["messages"][-1].pretty_print()
      else:
        if step['messages'][-1].response_metadata.get('finish_reason')=='stop':
          try:
            return display(Markdown(step['messages'][-1].content))
          except:
            return display(Markdown("I don't know"))

get_sql_reponse("Which is most profitable SKU_ID?")

The most profitable SKU_ID is 8, which corresponds to the product "t-shirt" in the "women" category, with a total profit of 693,898 INR.

In [20]:
get_sql_reponse("Which is most profitable SKU_ID?",verbose=True)

================================ Human Message =================================

Which is most profitable SKU_ID?
================================== Ai Message ==================================
Name: Retail_Data_Agent
Tool Calls:
  sql_db_schema (call_mmY27W49Z3vPzzEI7Vt2kUNb)
 Call ID: call_mmY27W49Z3vPzzEI7Vt2kUNb
  Args:
    table_names: historical_data
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE historical_data (
	sku_id INTEGER, 
	product_name TEXT, 
	category TEXT, 
	date TIMESTAMP, 
	unit_price REAL, 
	unit_cost REAL, 
	discount_pct REAL, 
	seasonality_factor REAL, 
	units_sold REAL, 
	revenue REAL, 
	profit REAL
)

/*
3 rows from historical_data table:
sku_id	product_name	category	date	unit_price	unit_cost	discount_pct	seasonality_factor	units_sold	revenue	profit
1	t-shirt	men	2023-06-01 00:00:00	1165.0	753.0	20.0	1.1	57.0	66405.0	23484.0
1	t-shirt	men	2023-07-01 00:00:00	1171.0	753.0	20.0	1.1	56.0	65576.

In [23]:
db.run('select sku_id, product_name,	category, sum(profit) profit from historical_data group by sku_id, product_name,	category;')

"[(1, 't-shirt', 'men', 656949.0), (8, 't-shirt', 'women', 693898.0)]"

In [24]:
get_sql_reponse("Which is most profitable SKU?")

The most profitable SKU is:
- **Product Name:** t-shirt
- **Category:** women
- **Total Profit:** ₹693,898.00

In [27]:
get_sql_reponse("Which is most profitable SKU of 2025?",verbose=True)

================================ Human Message =================================

Which is most profitable SKU of 2025?
================================== Ai Message ==================================
Name: Retail_Data_Agent
Tool Calls:
  sql_db_list_tables (call_EEypiOO91IQr5oIyF2ZoHarc)
 Call ID: call_EEypiOO91IQr5oIyF2ZoHarc
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

competitior_information, current_product_information, forecast_data, historical_data, inventory_data
================================== Ai Message ==================================
Name: Retail_Data_Agent
Tool Calls:
  sql_db_schema (call_YtAUP4zzcugLmzZ835O6wZEN)
 Call ID: call_YtAUP4zzcugLmzZ835O6wZEN
  Args:
    table_names: historical_data
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE historical_data (
	sku_id INTEGER, 
	product_name TEXT, 
	category TEXT, 
	date TIMESTAMP, 


In [28]:
db.run('select sku_id, product_name,	category, sum(profit) profit from historical_data where date >= "2025-01-01" group by sku_id, product_name,	category;')

"[(1, 't-shirt', 'men', 188747.0), (8, 't-shirt', 'women', 196767.0)]"

In [30]:
get_sql_reponse("Which is total profit of each SKU in year 2025?", verbose=True)

================================ Human Message =================================

Which is total profit of each SKU in year 2025?
================================== Ai Message ==================================
Name: Retail_Data_Agent
Tool Calls:
  sql_db_list_tables (call_FggjzY05MfjeTUtYUOP2NeTl)
 Call ID: call_FggjzY05MfjeTUtYUOP2NeTl
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

competitior_information, current_product_information, forecast_data, historical_data, inventory_data
================================== Ai Message ==================================
Name: Retail_Data_Agent
Tool Calls:
  sql_db_schema (call_O40ox5wDL0zb8YgbKzNGMgkP)
 Call ID: call_O40ox5wDL0zb8YgbKzNGMgkP
  Args:
    table_names: historical_data
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE historical_data (
	sku_id INTEGER, 
	product_name TEXT, 
	category TEXT, 
	date TI

In [31]:
get_sql_reponse("What is most elastic SKU?",verbose=True)

================================ Human Message =================================

What is most elastic SKU?
================================== Ai Message ==================================
Name: Retail_Data_Agent
Tool Calls:
  sql_db_list_tables (call_yp8tHt1qSnhhO9CtKa1Ydtae)
 Call ID: call_yp8tHt1qSnhhO9CtKa1Ydtae
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

competitior_information, current_product_information, forecast_data, historical_data, inventory_data
================================== Ai Message ==================================
Name: Retail_Data_Agent
Tool Calls:
  sql_db_schema (call_clCn7myEDSzTNx1aoA72Qhz2)
 Call ID: call_clCn7myEDSzTNx1aoA72Qhz2
  Args:
    table_names: current_product_information
  sql_db_schema (call_V22GsloBB2czwVgN7pPRBR7l)
 Call ID: call_V22GsloBB2czwVgN7pPRBR7l
  Args:
    table_names: historical_data
================================= Tool Message ================================

In [32]:
get_sql_reponse("compare our SKU's prices with competitor's prices?")

Here are the comparisons of our SKU's prices with competitor's prices:

1. **Product**: T-shirt (Men)
   - **Our Price**: ₹1460.0
   - **Competitor's Price**: ₹864.5

2. **Product**: T-shirt (Women)
   - **Our Price**: ₹1820.0
   - **Competitor's Price**: ₹946.4

These examples show that our prices are significantly higher than those of the competitors for both products.

In [41]:
from langchain.tools import tool

@tool("simulate_demand_tool", return_direct=True)
def simulate_demand_tool(elasticity, discount = 0.1, P1=100, Q1=1000):
    """
    Simulate new demand and revenue given price change and elasticity.

    Parameters:
        elasticity (float): Price elasticity of demand (negative)
        discount (float): Discount (in range 0-1)
        P1 (float): Initial price
        Q1 (float): Initial quantity

    Returns:
        dict with new quantity, new revenue, % change in Q, % change in revenue
    """
    P2 = P1 * (1+discount)
    # % change in quantity
    pct_change_Q = elasticity * discount

    # new quantity
    Q2 = Q1 * (1 + pct_change_Q)

    # revenues
    R1 = P1 * Q1
    R2 = P2 * Q2

    return {
        "Initial Price": P1,
        "New Price": P2,
        "Initial Quantity": Q1,
        "New Quantity": round(Q2, 2),
        "Initial Revenue": round(R1, 2),
        "New Revenue": round(R2, 2),
        "% Change Quantity": round(pct_change_Q * 100, 2),
        "% Change Revenue": round(((R2 - R1) / R1) * 100, 2)
    }

# # Example runs:
# print(simulate_demand(-0.6, -0.1))
# print(simulate_demand(-1.5, -0.1))

# print(simulate_demand(-0.6, 0.1))
# print(simulate_demand(-1.5, 0.1))

## Connect data agent with Main Supervisor Agent

In [33]:
!pip install -q langgraph-supervisor

In [34]:
from langgraph_supervisor import create_supervisor

In [56]:
from langgraph_supervisor import create_supervisor
from langchain.chat_models import init_chat_model

tools = [simulate_demand_tool]

supervisor = create_supervisor(
    name="Main_Retail_Agent",
    model=llm,
    tools=tools,
    agents=[data_agent],
    prompt=(
        "You are a supervisor agent responsible to build pricing strategy for a retail company:\n"
        "When you get a question, plan it in detail, check all the tools and agents you can use. then execute step by step.\n"
        "Use given data agent to assign any data related question to this agent and get answers\n"
        "Use simulate_demand_tool to check impact of a price change in any SKU. Do not do price change calculation by your own.\n"
        "Assign work to agents and build strategy using the data it returns.\n"
        "All the price data is in INR."
    ),
    add_handoff_back_messages=True,
    output_mode="full_history",
).compile()


### some utility function(ignore)

In [58]:
from langchain_core.messages import convert_to_messages



def pretty_print_messages(update, last_message=False):
    is_subgraph = False
    if isinstance(update, tuple):
        ns, update = update
        # skip parent graph updates in the printouts
        if len(ns) == 0:
            return

        graph_id = ns[-1].split(":")[0]
        print(f"Update from subgraph {graph_id}:")
        print("\n")
        is_subgraph = True

    for node_name, node_update in update.items():
        update_label = f"Update from node {node_name}:"
        if is_subgraph:
            update_label = "\t" + update_label

        print(update_label)
        print("\n")

        messages = convert_to_messages(node_update["messages"])
        if last_message:
            messages = messages[-1:]

        for m in messages:
            pretty_print_message(m, indent=is_subgraph)
        print("\n")

In [60]:
def chat_with_supervisor(input):
  for chunk in supervisor.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": input,
            }
        ]
    },
  ):
    pretty_print_messages(chunk, last_message=False)
  final_message_history = chunk["supervisor"]["messages"]


chat_with_supervisor("Which is our highest revenue generating item?")

Update from node supervisor:


================================ Human Message =================================

Which is our highest revenue generating item?
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_retail_data_agent (call_Uj7fkwL6z8JFmIm1VyfmOAdy)
 Call ID: call_Uj7fkwL6z8JFmIm1VyfmOAdy
  Args:
================================= Tool Message =================================
Name: transfer_to_retail_data_agent

Successfully transferred to Retail_Data_Agent


Update from node Retail_Data_Agent:


================================ Human Message =================================

Which is our highest revenue generating item?
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_retail_data_agent (call_Uj7fkwL6z8JFmIm1VyfmOAdy)
 Call ID: call_Uj7fkwL6z8JFmIm1VyfmOAdy
  Args:
================================= Tool Message ==================

In [61]:
chat_with_supervisor("How is its sale in next 3 months?")

Update from node supervisor:


================================ Human Message =================================

How is its sale in next 3 months?
================================== Ai Message ==================================
Name: supervisor

To analyze the sales in the next 3 months for a specific SKU, I need to gather relevant sales data and any other pertinent information that could affect sales, such as historical performance, seasonality, and market trends.

I'll start by transferring this task to the Retail_Data_Agent to obtain the necessary data regarding current sales trends, historical performance, and any other metrics that can inform our sales forecasts.

Let's proceed with that.
Tool Calls:
  transfer_to_retail_data_agent (call_WJSM1v4D4YtF0G5iZuXcinsX)
 Call ID: call_WJSM1v4D4YtF0G5iZuXcinsX
  Args:
================================= Tool Message =================================
Name: transfer_to_retail_data_agent

Successfully transferred to Retail_Data_Agent


Update 

In [62]:
# chat_with_supervisor("What will happen if we increase our sku_id 1's price by 10%?")

In [63]:
# chat_with_supervisor("What will happen if we decrease our sku_id 1's price by 10%? Describe in detail.")

In [64]:
chat_with_supervisor("I am considering a 10% price drop promotion in the store. What will be the impact on sales,revenue and profit? Do I have enough inventory for this? Describe in detail.")

Update from node supervisor:


================================ Human Message =================================

I am considering a 10% price drop promotion in the store. What will be the impact on sales,revenue and profit? Do I have enough inventory for this? Describe in detail.
================================== Ai Message ==================================
Name: supervisor

To assess the impact of a 10% price drop promotion on sales, revenue, and profit, we can follow these steps:

1. **Identify the SKU**: We need to specify which SKU (Stock Keeping Unit) the promotion will be applied to. Knowing the current price, quantity sold, and elasticity of demand for that SKU is essential.

2. **Gather Data**: 
   - Current price (P1)
   - Current quantity sold (Q1)
   - Price elasticity of demand for the SKU
   - Current inventory levels to assess if there are enough products to meet the expected increase in demand.

3. **Simulate Demand**: Using the `simulate_demand_tool`, we can analyze h

In [65]:
chat_with_supervisor("Let's say I filled the stock. Then what is the impact on profit?")

Update from node supervisor:


================================ Human Message =================================

Let's say I filled the stock. Then what is the impact on profit?
================================== Ai Message ==================================
Name: supervisor

To analyze the impact on profit after filling the stock, we need to consider:

1. **Current Price and Revenue:** Understand the current selling price (P1) and existing sales volume (Q1) for the SKU.
2. **Cost of Goods Sold (COGS):** Determine the cost associated with filling the stock, which will affect the profit margin.
3. **Potential Price Changes:** Any intended discount or price changes after restocking.
4. **Elasticity of Demand:** Understand how sensitive customers are to price changes, as this will help simulate changes in demand and revenue.

To gather this information, I will start by retrieving relevant sales data for the specific SKU from the Retail_Data_Agent. 

Let me proceed with that.
Tool Calls:
 

# We need to add memory and improve the quality of prompt to understand questions better

In [98]:
output = []
# input = "Which is our highest revenue generating item?"
# input = "How many SKU-1 are left in the inventory?"
input = "What if we increase price of SKU-1 by 10% ?"

for chunk in supervisor.stream(
  {
      "messages": [
          {
              "role": "user",
              "content": input,
          }
      ]
  },
):
  for k,v in chunk.items():
    print(k)
    for v1 in chunk[k]['messages']:
      print(v1.content)


supervisor
What if we increase price of SKU-1 by 10% ?

Successfully transferred to Retail_Data_Agent
Retail_Data_Agent
What if we increase price of SKU-1 by 10% ?

Successfully transferred to Retail_Data_Agent

competitior_information, current_product_information, forecast_data, historical_data, inventory_data


CREATE TABLE current_product_information (
	sku_id INTEGER, 
	base_price REAL, 
	base_demand REAL, 
	elasticity REAL, 
	margin REAL
)

/*
3 rows from current_product_information table:
sku_id	base_price	base_demand	elasticity	margin
1	1460.0	500.0	-1.06	0.94
8	1820.0	800.0	-0.93	1.08
*/


CREATE TABLE forecast_data (
	sku_id INTEGER, 
	product_name TEXT, 
	category TEXT, 
	date TIMESTAMP, 
	units_sale REAL
)

/*
3 rows from forecast_data table:
sku_id	product_name	category	date	units_sale
1	t-shirt	men	2025-09-01 00:00:00	41.0
1	t-shirt	men	2025-10-01 00:00:00	41.0
1	t-shirt	men	2025-11-01 00:00:00	41.0
*/


CREATE TABLE historical_data (
	sku_id INTEGER, 
	product_name TEXT, 

Data agent is doing most of the work, need to teach main agent to pass only relevant questions to data agent and do rest of the work